# Formatting data

The aim of this notebook is to download the data described in [this publication](https://www.nature.com/articles/s41586-020-03171-x) and store parts of it in an easily accessible format.
For each recording session, there will be three output files:
- `units.cvs`: Table listing all recorded units and a bunch of metrics
- `stimuli.csv`: Table listing all occurrences of the selected stimuli for the session
- `spike_times.npy`: The spike times for every unit recorded in that session
- `mean_waveforms.npy`: Mean waveform of every unit recorded in that session


In [ ]:
from pathlib import Path
from typing import List
import pandas as pd
import numpy as np
from pynwb import NWBHDF5IO, NWBFile
from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache

## Set up parameters
Select session and regions of interest

In [ ]:
root = Path('/home/obi/projects/Publication-Ready-Figures-With-Matplotlib')
cache = EcephysProjectCache.from_warehouse(manifest=root/"data"/"raw"/"manifest.json")
SESSION_TYPE="brain_observatory_1.1"
ROI = ["VISpm", "VISrl", "VISam"] # primary, rostrolateral and anteriormedial visual cortex
STIMULI = ["gabors", "flashes", "drift_gratings", "static_gratings", "natural_scenes"]
GENOTYPES = ["wt/wt"]

## Filter Sessions
Select the sessions with the given `GENOTYPES` and `SESSION_TYPE`

In [ ]:
sessions = cache.get_session_table()
sessions = sessions[sessions.session_type==SESSION_TYPE]
sessions = sessions[sessions.full_genotype.isin(GENOTYPES)]

## Download data
Download the data to the cache folder

In [ ]:
#| label: session-loop
session_id = sessions.index[0]
for session_id in sessions.index:
    session = cache.get_session_data(session_id)

## Filter units
Select units located in ROI

In [ ]:
    units = session.units
    unit_in_ROI = []
    for i in units.index:
        structure = units.loc[i].ecephys_structure_acronym
        if structure in ROI:
            unit_in_ROI.append(True)
        else:
            unit_in_ROI.append(False)
    units = units[unit_in_ROI]

## Mean waveforms
Get the mean waveforms for the selected unit.
Because the mean waveforms are computed for every electrode, we only select the one with the largers peak-to-peak amplitude (should be from the electrode closest to the unit)

In [ ]:
    mean_waveforms = session.mean_waveforms
    mean_waveforms_out = {}
    mean_waveforms_out["time"] = mean_waveforms[units.index[0]].time.data
    for i in units.index:
        w = mean_waveforms[i].data
        ptp = w.max(axis=1)-w.min(axis=1)
        idx = np.argmax(ptp) # waveform with largest peak-to-peak ampitude
        mean_waveforms_out[i] = w[idx, :]

## Spike times
Get spike times for the selected units

In [ ]:
    spike_times = session.spike_times
    spike_times_out = {}
    for i in units.index:
        spike_times_out[i] = spike_times[i]

## Stimuli
Load stimulus table and select the ones defined in `STIMULI`

In [ ]:
    stimuli = session.get_stimulus_table()
    stimuli = stimuli[stimuli.stimulus_name.isin(STIMULI)]

## Write data
Now we can create an output folder and store the data

In [ ]:
    out_dir = root/"data"/f"ses_{session_id}"
    if not out_dir.exists():
        out_dir.mkdir()
    units.to_csv(out_dir/f"ses_{session_id}_units.csv")
    stimuli.to_csv(out_dir/f"ses_{session_id}_stimuli.csv")
    np.save(out_dir/f"ses_{session_id}_mean_waveforms.npy", mean_waveforms_out, allow_pickle=True)
    np.save(out_dir/f"ses_{session_id}_spike_times.npy", spike_times_out, allow_pickle=True)